In [22]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [23]:
from tqdm.auto import tqdm

In [24]:
import json

In [25]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [26]:
documents=[]
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']#adding course to each document 
        documents.append(doc)

In [27]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 7.1 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.4 MB/s eta 0:00:00.0 MB/s eta 0:00:01


In [28]:
from elasticsearch import Elasticsearch

In [29]:
es_client=Elasticsearch('http://localhost:9200')

In [30]:
es_client.info()#to check things work

ObjectApiResponse({'name': '08ef21d41d01', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5qbDZQIGR66Z63DiOaxmwQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [31]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name="course-questions"
es_client.indices.create(index=index_name,body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/uTPERkloQjacr_t66OoT1g] already exists')

#indexing the data

In [14]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [20]:
#The best_fields type searches each field separately and returns the highest score from any one of the fields.

#This type is useful when you want to find documents where at least one field matches the query well.

In [15]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [16]:
def build_prompt(query,search_results):
    prompt_template="""
    you are a course teaching assistant.answer the QUESTION  based on the CONTEXT from the FAQ database.
    QUESTION:{question}

    CONTEXT:{context}""".strip()
    context=""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt):
    response=client.chat.completions.create(
    model='phi3',
    messages=[{"role":"user","content":prompt}]
    )
    return response.choices[0].message.content

In [18]:
from openai import OpenAI

In [19]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [20]:
def rag(query):
    search_results=elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer
    

In [21]:
query = 'I just disovered the course. Can I still join it?'
rag(query)

InternalServerError: Error code: 500 - {'error': {'message': 'model requires more system memory (5.6 GiB) than is available (4.4 GiB)', 'type': 'api_error', 'param': None, 'code': None}}